### Introduction

- Language Model Scoring하는 것에 사용을 하며, Scoring을 한다는 것은 Sentence generation을 할 수 있다는 얘기로도 볼 수 있다. 
- Seq2Seq Model 또는 Neural Machine Translation에 대해 살펴보자.


- Machine Translation: 주어진 문장을 원하는 언어의 문장으로 자동으로 바꾸어주는 것.
- Neural Net을 사용하면서 Machine Translation이 좁은 의미에서 의미가 확장되었다. 주어진 input에 대해 전혀 다른 structure가 있는 sequence로 바꾸어줄 수 있는가 로 Generalize 되었다.


- Machine Translation의 역사를 보고, Neural Net의 사용이 주는 이점을 보자.


## A bit of historical remark

<img  src="./image/img_6-1.PNG" width="55%">

- 실제 우리가 translate하는 방법을 모방하려고 노력.
- word structure $ \to $ syntactic $ \to $ semantic $ \to $ interlingua(모든 언어의 기반이 되는 그런 의미) $ \to $ semantic $ \to $ syntactic $ \to $ word structure


- BUT 이 Rule-Based System이 희망이 안보이고, 30년 동안 큰 진전이 없었다.
- 그래서 IBM의 Fred Jelineak님이 통계와 정보이론을 활용해 문제를 풀어보겠다. 하며 다음과 같은 명언을 날린다. "Every time i fire a linguist the performance of the speech recognizer goes up." 
    - 나중에 이런 말 안했다고 하긴 했지만, 들은 사람이 한둘이 아니었다.
     
<img  src="./image/img_6-2.PNG" width="55%">    

- Rule based은 Narrow한 도메인에서 사용하긴한다
- Statistical MT (ML을 사용한 것이) 주류가 되어 IBM Models 등등 쭉 다양한 시스템이 나오고, 2000년대 중반부터 지금(Google Translation)까지 Large-scale MT system을 만드는데 이용이 되었다.
- 근데 IBM Model들이 쭉 나올 때도 뉴럴 모델을 사용하려는 사람들이 이미 있었다.


- 그리고 초기 뉴럴 모델들 설명, 지금과 비슷한 생각을 갖고 있었지만 컴퓨팅 능력의 한계로 Narrow하게 됨을 보이기만 했다.
    - [Allen 1987 IEEE 1st ICNN], [Chrisman 1992 Connection Science]
    - [Focada & Neco, 1997], [Castano & Casacuerta, 1997]
        - Focada와 Castano가 맥주를 마시며 뉴럴넷을 써바야겠다해서 쓴 다른 두 논문
        - 이 때 이후로 Focada는 NN버리고 Rule-based MT로 쭉 해와서 굉장히 유명해짐. 최근에는 NN을 씀


- NN모델은 2000년대에 computing resource와 데이터들이 좋아지면서 나오게 된다.

<img  src="./image/img_6-3.PNG" width="55%">  

- [Schwenk et al. 2006] : 지지난 시간에 배운 Neural N-gram Language Model을 사용해 프랑스 Facebook AI Research에서 , IBM Model이나 Phrase-based Statistical에서 나온 결과들을(K-best List) Neural Net Language Model을 사용해 Rescoring하자.
    - IBM Model이나 Phrase-based Statistical에서 나온 결과(K-best List)가 그닥 좋아보이지 않아, Neural N-gram Language Model을 활용해 K-best 중에서 가장 보기 좋은 translation을 선택하는 모델.
    - 혼자만하고 많이들 따라하지 못했다. Resource환경 때문에
    
- [Devlin et al. 2014] : 현재 google의 연구원이 NN을 Statistical MT system안에 넣어 버리자. 해서 결과가 확 좋아진다 함.

- 그리고, 한두해 지나고 나서 이제 맨 오른쪽 다 뉴럴 넷으로...
    - NN도 크게 할 수 있고 GPU도 좋고 하니 다 버리고 NN만 쓰자.
    - Modern MT가 됨.
    
<br>


## Machine Translation 

- Supervised Learning, Classification과 비슷하다. 다른 점이라면, input에 대해 output이 어떤 클래스에 속하느냐가 아니라 input에 대해 어떤 sentence가  translation이냐고 하는 것이다. 아래 식! $ p(Y|X) $

- Input: a sentence written in a source language : ** $ L_{S} $ **
- Output: a corresponding sentence in a target language : ** $ L_{T} $ **
- Problem statement:
    - Supervised learning: given the input sentence, output its translation
    - Compute the conditional distribution over all possible translation given the input
    - $ p(Y = ( y_{1}, \cdots, y_{T} ) | X = ( x_{1}, \cdots, x_{T'} ) ) $
    
- We have already learned every necessary ingredient for building a full neural machine translation system.
    - Scoring하는 Language 모델과 Sentence Representation을 잘 연결하면 된다!
    
    
### Token Representation – One-hot Vectors

- Text Classification, Language Model과 같이 Vocab을 만들자. 근데 MT는 Source와 Target 2개가 필요.
- 그러면 one hot vector가 남고 Table Look-up을 통해 Continuous Vector로 바뀌게


1. Build source and target vocabularies of unique tokens
    - For each of source and target languages,<br>
        - Tokenize: separate punctuations, normalize punctuations, ... using Spacy.io, NLTK or Moses’ tokenizer.<br>
        e.g., "I'm going" => ("I", "'m", "going"), replace ',',', into "'", ... 
        - Subword segmentation: segment each token into a sequence of subwords <br>
        e.g., “going” => (“go”, “ing”), use BPE [Sennrich et al., 2015] <br>
            - BPE 인코딩을 통해 하위 단어의 세분화를 진행
        - Collect all unique subwords, sort them by their frequencies (descending) and assign indices.
            - 모든 하위 단어를 통합하여 빈도수 내림차순으로 정렬한 뒤 인덱스 부여
            
2. Transform each subword token into a corresponding one-hot vector.*
    - See Lecture 2.
    
    
### Encoder – Source Sentence Representation
- Encode the source sentence into a set of sentence representation vectors
    - number of encoded vectors is proportional to the source sentence length: often same.
    - Recurrent networks have been widely used [Cho et al., 2014; Sutskever et al., 2014], but CNN [Gehring et al., 2017; Kalchbrenner&Blunsom, 2013] and self-attention [Vaswani et al., 2017] are used increasingly more often. See Lecture 2 for details.
- We do not want to collapse them into a single vector!
    - Collapsing often corresponds to information loss.
    - Increasingly more difficult to encode the entire source sentence into a single vector, as the sentence length increases [Cho et al., 2014b].
    - We didn’t know initially until [Bahdanau et al., 2015].
    
    
- sentence representation의 방법 중 고르면 되는데,  Text Classification 할 때와 다른점은 굳이 얘를 fixed-size할 필요는 없다. 사실 지금에와서야 이렇게 말하는 거지 교수님이랑 구글 브레인도 사이즈를 맞춰야하는 줄 알았다. 안하는게 좋은 이유는
    - 하나의 벡터로 합치게 되면 정보 손실이 생긴다.
    - 문장의 길이가 길어질수록 훈련하기도 힘들고 모델의 크기를 키워야한다.

<img  src="./image/img_6-4.PNG" width="45%">  

- 조경현 교수님도 vector를 하나로 합치고 할 때, NN을 사용한 MT의 한계점으로 알았던 논문 내용. 
- BLEU는 100점 만점으로 사람이 translation하면 보통 40~50점 나옴. 70단어 이상의 문장은 말도 안되는 문장으로 번역한다.
    - 나중에 보니 하나의 벡터로로 합친 것이 문제인줄 알게됨.


- Collapse의 할 때 생긴 문제점 예시
    - Source: An admitting privilege is the right of a doctor to admit a patient to a hospital or a medical centre <u> to carry out a diagnosis or a procedure, based on his status as a health care worker at a hospital. </u> 
    - When collapsed: Un privilège d’admission est le droit d’un médecin de reconnaître un patient à l’hôpital ou un centre médical <u> d’un diagnostic ou de prendre un diagnostic en fonction de son état de santé. </u>    
    - 영어 => 불어로 할 때, 앞부분은 거의 완벽하게 했으나, 뒷 부분으로 가서는 모든 정보를 하나의 벡터에 넣지 못하다 보니까, 어느 순간 Language Modelling을 한다.
    - 지금까지 내가 이런 말을 했고 information은 Loss됬고 Source Sentence도 모르니까, 그냥 내 멋데로 말해야겠다. 그래서 문법적으로는 맞지만 번역 내용과는 완전히 다른 output을 냈다.
    
###    Decoder – Language Modelling

- 그래서 Collapse를 안하고, Token representation을 token 개수 만큼 다 가져간다.
- 그리고 실제로 translation을 generate하는 부분은 Language Model을 사용할 것이다.
    - 여기서 중요한 부분은 Autoregressive Language model을 사용한다는 것과 Context를 무한대로 보겠다는 것.
    - "N-gram으로 N을 정해서 앞의 한 3단어를 보고 다음단어를 정해야지" 하는게 더 이상 안되는 것. 
    - 이것이 scoring과 sentence generation의 차이이다. 문법과 같은 것을 고려해야한다. garden path sentence을 방지 해야하니까.
    

- <u> Autoregressive Language modelling with an infinite context </u>  $ n \to \infty $
    - Larger context is necessary to generate a coherent sentence.
        - Semantics could be largely provided by the source sentence, but syntactic properties need to be handled by the language model directly.
    - Recurrent networks, self-attention and (dilated) convolutional networks
        - Causal structure must be followed.
        - See Lecture 3.
        - 배운 것들 중 사용할 것은 3개가 남게된다. 
        
- ** Conditional ** Language modelling
    - The context based on which the next token is predicted is ** two-fold **
    - $ p(Y|X) = \prod_{t=1}^{T} p(y_{t}|y_{<t}, X )  $
    - Conditional하게 만드는 것이 달라지는 점이다. Supervised Learning으로 Language 모델로 만든 것은 기존에 나왔던 토큰을 보고 다음 토큰을 predict하는 것이었는데
    - 이제는 input에 하나 더 추가해, 기존에 나왔던 토큰들을 더해서 Source sentence가 무엇인지 같이 봐서 다음 단어가 무엇인지를 결정해 주는 방법으로 살짝 틀어야한다. 
    - $ | y_{<t}, X $ 이 부분
    
- Supervised learning: T input-output training pairs per sentence
    - Input: the entire source sentence : $ X $ and the preceding target tokens : $ y<t $
    - Output: the next token $ y_{t} $
    
- 그래도 거의 달라지는 것이 거의 없이 Text Classification을 또 똑같이 하는 것과 같다.
     
<img  src="./image/img_6-5.PNG" width="60%">      

- 지금까지 한 것과 같지만 물음표가 조금 복잡해질 것 같다.    
    
    

### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 

### Papers:
- BPE Encoding: https://arxiv.org/abs/1508.07909